###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de https://www.datos.gov.co y corresponden a datos cuya fuente primaria es el Sistema de Información de Eventos de Violencia del Conflicto Armado SIEVCAC

In [48]:
import pandas as pd

### Descarga de set de datos y carga como df

Se descarga 11 conjuntos de datos que provienen del Sistema de Información de Eventos de Violencia del Conflicto Armado en Colombia (SIEVCAC) y recopilan datos sobre las Casos documentados del conflicto armado, donde se han presentado victimas, clasificados en un tipo específico de violencia: desaparición forzada, atentados terroristas, secuestro, asesinatos selectivos, víctimas de minas antipersonal, violencia sexual, daño a bienes civiles, masacres, acciones bélicas, ataques a poblados y reclutamiento de menores

In [49]:
# Lista urls victimas del conflicto -
cases_dataset_urls = [
     ("casos_desaparicion_forzada", "https://www.datos.gov.co/resource/9r7i-ek3s.csv"),
     ("casos_atentado_terrorista", "https://www.datos.gov.co/resource/yfd7-8c9d.csv"),
     ("casos_secuestro", "https://www.datos.gov.co/resource/cdw6-jexv.csv"),
     ("casos_asesinatos_selectivos", "https://www.datos.gov.co/resource/sysu-pezb.csv"),
     ("casos_minas", "https://www.datos.gov.co/resource/grup-7tjb.csv"),
     ("casos_violencia_sexual", "https://www.datos.gov.co/resource/k9qg-2svz.csv"),
     ("casos_dano_bienes_civiles", "https://www.datos.gov.co/resource/x27p-cm5t.csv"),
     ("casos_masacres", "https://www.datos.gov.co/resource/d78j-f66e.csv"),
     ("casos_acciones_belicas", "https://www.datos.gov.co/resource/39qq-a72j.csv"),
     ("casos_ataques_a_poblados", "https://www.datos.gov.co/resource/3xsy-8xcp.csv"),
     ("casos_reclutamiento_ninos", "https://www.datos.gov.co/resource/hzd2-7ea7.csv"),

 ]

In [50]:
# Función para descargar datasets y almacenarlos en un diccionario
def download_datasets(dataset_urls):
    datasets = {}
    for name, url in dataset_urls:
        try:
            # Descargar el dataset y cargarlo en un DataFrame
            df = pd.read_csv(url+'?$limit=1000000')
            # Almacenar el DataFrame en el diccionario con el nombre del dataset
            datasets[name] = df
            print(f"Descargado y almacenado en memoria: {name}")
        except Exception as e:
            print(f"Error al descargar {name}: {str(e)}")
    return datasets


In [51]:
# Llamar a la función y almacenar los datasets en un diccionario
datasets = download_datasets(cases_dataset_urls)

Descargado y almacenado en memoria: casos_desaparicion_forzada
Descargado y almacenado en memoria: casos_atentado_terrorista
Descargado y almacenado en memoria: casos_secuestro
Descargado y almacenado en memoria: casos_asesinatos_selectivos
Descargado y almacenado en memoria: casos_minas
Descargado y almacenado en memoria: casos_violencia_sexual
Descargado y almacenado en memoria: casos_dano_bienes_civiles
Descargado y almacenado en memoria: casos_masacres
Descargado y almacenado en memoria: casos_acciones_belicas
Descargado y almacenado en memoria: casos_ataques_a_poblados
Descargado y almacenado en memoria: casos_reclutamiento_ninos


In [52]:
# cambiamos nombre a columna de un dataset para que todos tengan el mismo nombre

# Accediendo al dataset
df = datasets['casos_desaparicion_forzada']

# Cambiando el nombre de la columna
df = df.rename(columns={'codigo_dane_de_municipio': 'c_digo_dane_de_municipio'})
df = df.rename(columns={'ano': 'a_o'})

# Actualizando el dataset en el diccionario
datasets['casos_desaparicion_forzada'] = df
datasets['casos_desaparicion_forzada'].columns

Index(['id_caso', 'id_caso_relacionado', 'a_o', 'mes', 'dia',
       'c_digo_dane_de_municipio', 'municipio', 'departamento', 'region',
       'modalidad', 'presunto_responsable', 'descripci_n_presunto',
       'abandono_o_despojo_forzado', 'amenaza_o_intimidacion',
       'ataque_contra_mision_medica', 'confinamiento_o_restriccion',
       'desplazamiento_forzado', 'extorsion', 'lesionados_civiles', 'pillaje',
       'tortura', 'violencia_basada_en_genero', 'otro_hecho_simult_neo',
       'total_de_v_ctimas_del_caso', 'tipo_de_armas', 'latitud_longitud'],
      dtype='object')

In [53]:
# Función para agregar una columna 'tipo_de_crimen' a cada dataset
def add_type_of_crime(datasets):
    for name, df in datasets.items():
        # Remover 'victimas_' del nombre del dataset
        tipo_crimen = name.replace('casos_', '')
        # Agregar la nueva columna al DataFrame
        df['tipo_de_crimen'] = tipo_crimen
        # Actualizar el DataFrame en el diccionario
        datasets[name] = df

    return datasets

In [54]:
# Ejecutar funcion add_type_of_crime
datasets = add_type_of_crime(datasets)

# Verificar que la columna fue agregada en un dataset
datasets['casos_asesinatos_selectivos'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154315 entries, 0 to 154314
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   id_caso                      154315 non-null  int64 
 1   id_caso_relacionado          5619 non-null    object
 2   a_o                          154315 non-null  int64 
 3   mes                          154315 non-null  int64 
 4   d_a                          154315 non-null  int64 
 5   c_digo_dane_de_municipio     154315 non-null  int64 
 6   municipio                    154315 non-null  object
 7   departamento                 154315 non-null  object
 8   regi_n                       153685 non-null  object
 9   modalidad                    154315 non-null  object
 10  presunto_responsable         154315 non-null  object
 11  descripci_n_presunto         154313 non-null  object
 12  abandono_o_despojo_forzado   154315 non-null  int64 
 13  amenaza_o_inti

In [55]:
# Función para obtener las columnas comunes a todos los datasets
def common_columns(datasets):
    # Extraer las columnas del primer dataset como base
    common_columns = set(datasets[next(iter(datasets))].columns)

    # Iterar sobre los demás datasets y realizar la intersección de columnas
    for name, df in datasets.items():
        common_columns &= set(df.columns)  # Intersección de conjuntos

    return list(common_columns)  # Retornar el resultado como lista

In [56]:
# Obtener las columnas comunes a todos los datasets
all_common_columns = common_columns(datasets)

# Mostrar las columnas comunes
print("Columnas comunes a todos los datasets:", all_common_columns)

Columnas comunes a todos los datasets: ['a_o', 'id_caso_relacionado', 'id_caso', 'c_digo_dane_de_municipio', 'lesionados_civiles', 'tipo_de_crimen', 'municipio', 'mes', 'total_de_v_ctimas_del_caso', 'departamento']


In [57]:
len(all_common_columns)

10

In [58]:
# Funcion para filtar solamente columnas comunes
def filter_common_columns(datasets, common_columns):
    for name, df in datasets.items():
        # Filtrar el DataFrame para que solo contenga las columnas en common_columns
        datasets[name] = df.filter(items=common_columns)
    return datasets

In [59]:
# Ejemplo de uso
filtered_datasets = filter_common_columns(datasets, all_common_columns)

# Verificar que los DataFrames han sido filtrados
for name, df in filtered_datasets.items():
    print(f"\n{ name }:\n", df.columns)


casos_desaparicion_forzada:
 Index(['a_o', 'id_caso_relacionado', 'id_caso', 'c_digo_dane_de_municipio',
       'lesionados_civiles', 'tipo_de_crimen', 'municipio', 'mes',
       'total_de_v_ctimas_del_caso', 'departamento'],
      dtype='object')

casos_atentado_terrorista:
 Index(['a_o', 'id_caso_relacionado', 'id_caso', 'c_digo_dane_de_municipio',
       'lesionados_civiles', 'tipo_de_crimen', 'municipio', 'mes',
       'total_de_v_ctimas_del_caso', 'departamento'],
      dtype='object')

casos_secuestro:
 Index(['a_o', 'id_caso_relacionado', 'id_caso', 'c_digo_dane_de_municipio',
       'lesionados_civiles', 'tipo_de_crimen', 'municipio', 'mes',
       'total_de_v_ctimas_del_caso', 'departamento'],
      dtype='object')

casos_asesinatos_selectivos:
 Index(['a_o', 'id_caso_relacionado', 'id_caso', 'c_digo_dane_de_municipio',
       'lesionados_civiles', 'tipo_de_crimen', 'municipio', 'mes',
       'total_de_v_ctimas_del_caso', 'departamento'],
      dtype='object')

casos_minas:
 

In [60]:
# Integrar verticalmente todos los datasets
# Integrar todos los DataFrames filtrados verticalmente
all_datasets = pd.concat(filtered_datasets.values(), ignore_index=True)

# Verificar la estructura del DataFrame integrado
all_datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360220 entries, 0 to 360219
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   a_o                         360220 non-null  int64  
 1   id_caso_relacionado         24116 non-null   object 
 2   id_caso                     360220 non-null  int64  
 3   c_digo_dane_de_municipio    360220 non-null  int64  
 4   lesionados_civiles          360220 non-null  float64
 5   tipo_de_crimen              360220 non-null  object 
 6   municipio                   360220 non-null  object 
 7   mes                         360220 non-null  int64  
 8   total_de_v_ctimas_del_caso  360220 non-null  float64
 9   departamento                360220 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 27.5+ MB


In [61]:
# Ajuste nombre dataset
cases_armed_conflict_sievcac = all_datasets.copy()
cases_armed_conflict_sievcac.head()

,a_o,id_caso_relacionado,id_caso,c_digo_dane_de_municipio,lesionados_civiles,tipo_de_crimen,municipio,mes,total_de_v_ctimas_del_caso,departamento
0,1991,NaN,100265,25368,0.0,desaparicion_forzada,JERUSALEN,5,1.0,CUNDINAMARCA
1,2004,NaN,100282,25386,0.0,desaparicion_forzada,LA MESA,12,1.0,CUNDINAMARCA
2,1993,CR000088,101616,25885,0.0,desaparicion_forzada,YACOPI,3,2.0,CUNDINAMARCA
3,1997,NaN,102675,27615,0.0,desaparicion_forzada,RIOSUCIO,5,1.0,CHOCO
4,2006,NaN,103705,41396,0.0,desaparicion_forzada,LA PLATA,11,2.0,HUILA


In [62]:
columns = cases_armed_conflict_sievcac.columns

In [63]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {cases_armed_conflict_sievcac[col].nunique()}')

a_o: 66
id_caso_relacionado: 13964
id_caso: 360220
c_digo_dane_de_municipio: 1151
lesionados_civiles: 63
tipo_de_crimen: 11
municipio: 1042
mes: 13
total_de_v_ctimas_del_caso: 70
departamento: 35


In [64]:
# Eliminar columnas que se considera innecesarias para el proyecto
important_columns = ['id_caso', 'departamento', 'municipio', 'c_digo_dane_de_municipio','a_o', 'tipo_de_crimen', 'lesionados_civiles', 'total_de_v_ctimas_del_caso' ]
cases_armed_conflict_sievcac = cases_armed_conflict_sievcac[important_columns]
cases_armed_conflict_sievcac.head()

,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,lesionados_civiles,total_de_v_ctimas_del_caso
0,100265,CUNDINAMARCA,JERUSALEN,25368,1991,desaparicion_forzada,0.0,1.0
1,100282,CUNDINAMARCA,LA MESA,25386,2004,desaparicion_forzada,0.0,1.0
2,101616,CUNDINAMARCA,YACOPI,25885,1993,desaparicion_forzada,0.0,2.0
3,102675,CHOCO,RIOSUCIO,27615,1997,desaparicion_forzada,0.0,1.0
4,103705,HUILA,LA PLATA,41396,2006,desaparicion_forzada,0.0,2.0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [65]:
cases_armed_conflict_sievcac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360220 entries, 0 to 360219
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id_caso                     360220 non-null  int64  
 1   departamento                360220 non-null  object 
 2   municipio                   360220 non-null  object 
 3   c_digo_dane_de_municipio    360220 non-null  int64  
 4   a_o                         360220 non-null  int64  
 5   tipo_de_crimen              360220 non-null  object 
 6   lesionados_civiles          360220 non-null  float64
 7   total_de_v_ctimas_del_caso  360220 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 22.0+ MB


In [66]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'tipo_de_crimen']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(cases_armed_conflict_sievcac[column].unique())
    print()

Categorías en la columna 'departamento':
['CUNDINAMARCA' 'CHOCO' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA'
 'CASANARE' 'PUTUMAYO' 'ANTIOQUIA' 'BOLIVAR' 'CAQUETA' 'CAUCA' 'CESAR'
 'CORDOBA' 'CALDAS' 'EXTERIOR' 'GUAVIARE' 'ARAUCA' 'RISARALDA' 'QUINDIO'
 'BOYACA' 'BOGOTA, D. C.' 'ATLANTICO' 'SIN INFORMACION' 'VICHADA' 'VAUPES'
 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA']

Categorías en la columna 'municipio':
['JERUSALEN' 'LA MESA' 'YACOPI' ... 'LA PEDRERA (ANM)' 'FRONTERA PERÚ'
 'GACHANTIVA']

Categorías en la columna 'tipo_de_crimen':
['desaparicion_forzada' 'atentado_terrorista' 'secuestro'
 'asesinatos_selectivos' 'minas' 'violencia_sexual' 'dano_bienes_civiles'
 'masacres' 'acciones_belicas' 'ataques_a_poblados' 'reclutamiento_ninos']



In [67]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in categorical_col:
  l = cases_armed_conflict_sievcac[col].nunique()
  if l < 50:
    print(f'{col}: {cases_armed_conflict_sievcac[col].unique()}')

departamento: ['CUNDINAMARCA' 'CHOCO' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA'
 'CASANARE' 'PUTUMAYO' 'ANTIOQUIA' 'BOLIVAR' 'CAQUETA' 'CAUCA' 'CESAR'
 'CORDOBA' 'CALDAS' 'EXTERIOR' 'GUAVIARE' 'ARAUCA' 'RISARALDA' 'QUINDIO'
 'BOYACA' 'BOGOTA, D. C.' 'ATLANTICO' 'SIN INFORMACION' 'VICHADA' 'VAUPES'
 'GUAINIA' 'AMAZONAS'
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA']
tipo_de_crimen: ['desaparicion_forzada' 'atentado_terrorista' 'secuestro'
 'asesinatos_selectivos' 'minas' 'violencia_sexual' 'dano_bienes_civiles'
 'masacres' 'acciones_belicas' 'ataques_a_poblados' 'reclutamiento_ninos']


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [68]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [69]:
# Aplicar la función a todas las columnas categóricas
cases_armed_conflict_sievcac[categorical_col] = cases_armed_conflict_sievcac[categorical_col].astype(str)
for col in categorical_col:
    cases_armed_conflict_sievcac[col] = cases_armed_conflict_sievcac[col].apply(remove_accents_and_special_chars)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios estan en formato int64


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [70]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [71]:
cases_armed_conflict_sievcac.columns

Index(['id_caso', 'departamento', 'municipio', 'c_digo_dane_de_municipio',
       'a_o', 'tipo_de_crimen', 'lesionados_civiles',
       'total_de_v_ctimas_del_caso'],
      dtype='object')

In [72]:
# valores codigo dane del municipio
cases_armed_conflict_sievcac["c_digo_dane_de_municipio"].unique()

array([25368, 25386, 25885, ..., 77004, 15293, 97000])

- Longitud de los códigos municipios en el df

In [73]:
# Convertir a strings
cases_armed_conflict_sievcac["c_digo_dane_de_municipio"] = cases_armed_conflict_sievcac["c_digo_dane_de_municipio"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = cases_armed_conflict_sievcac["c_digo_dane_de_municipio"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[5 4 1] caracteres


In [74]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,c_digo_dane_de_municipio,count,percentage
0,5,249928,0.693820
1,4,101571,0.281969
2,1,8721,0.024210


In [75]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = cases_armed_conflict_sievcac[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['c_digo_dane_de_municipio']])
    print()

Muestra de registros con longitud 5:
  c_digo_dane_de_municipio
0                    25368
1                    25386
2                    25885
3                    27615
4                    41396

Muestra de registros con longitud 4:
   c_digo_dane_de_municipio
49                     5361
50                     5361
51                     5480
52                     5480
53                     5649

Muestra de registros con longitud 1:
    c_digo_dane_de_municipio
336                        0
337                        0
338                        0
339                        0
340                        0



Nota: De lo anterior se notan claras inconsistencias en la columna 'c_digo_dane_de_municipio' del df, el 69.4% tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 28.2% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió), un 2.4% tienen un codigo de un digito, que al parecer es el 0, y existe uno registro con un codigo de 6 digitos: 770000

- Verificar si los codigo_dane de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [76]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = cases_armed_conflict_sievcac[cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 4]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [77]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
cases_armed_conflict_sievcac['c_digo_dane_de_municipio'] = cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].apply(add_zero_if_length_4)

In [78]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(cases_armed_conflict_sievcac[cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 4])

0

- Resolver caso con codigo de 1 digito

In [79]:
# Imprimir una muestra
code_1_digit = cases_armed_conflict_sievcac[cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() == 1]
code_1_digit.head()

,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,lesionados_civiles,total_de_v_ctimas_del_caso
336,205568,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,0.0,1.0
337,205569,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,0.0,1.0
338,205570,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,0.0,1.0
339,205571,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,0.0,1.0
340,205572,SIN INFORMACION,SIN INFORMACION,0,0,DESAPARICION_FORZADA,0.0,1.0


In [80]:
# Verificar ninguno cuenta con informacion de departamento y municipio
print(code_1_digit['departamento'].unique())
print(code_1_digit['municipio'].unique())

['SIN INFORMACION']
['SIN INFORMACION']


In [81]:
# Se decide elimnar dichos registros porque son casos sin informacion, y no existe posibilidad de actualizarlos
cases_armed_conflict_sievcac = cases_armed_conflict_sievcac[cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].str.len() != 1]

In [82]:
cases_armed_conflict_sievcac.shape

(351499, 8)

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [83]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [84]:
# Comparar listas de códigos
compare_lists(cases_armed_conflict_sievcac['c_digo_dane_de_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en cases_armed_conflict_sievcac", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en cases_armed_conflict_sievcac que no están en Códigos de municipios dept_mpios_codes:
{'11000', '63000', '77001', '50000', '18000', '77004', '23000', '70000', '52000', '76000', '86000', '77002', '19000', '95000', '15000', '97000', '91000', '77005', '88000', '81000', '25000', '77000', '85000', '44000', '17000', '77003', '68000', '66000', '27086', '73000', '13000', '94663', '94000', '99000', '47000', '20000', '08000', '54000', '41000', '27000', '05000'}


In [85]:
# codigos que no corresponden a codigos reales de municipios
codes_wront = {'73000', '27000', '20000', '11000', '19000', '05000', '08000', '77003', '66000', '23000', '15000', '70000', '27086', '77005', '63000', '77000', '68000', '95000', '44000', '85000', '91000', '25000', '41000', '94000', '76000', '97000', '77001', '17000', '77004', '13000', '54000', '81000', '50000', '52000', '77002', '18000', '88000', '99000', '86000', '94663', '47000'}


In [86]:
# Imprimir una muestra
registers_confirm = cases_armed_conflict_sievcac[cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].isin(codes_wront)]
print(f'NO. registros con codigo errado: {len(registers_confirm)}')
registers_confirm.head()

NO. registros con codigo errado: 2031


,id_caso,departamento,municipio,c_digo_dane_de_municipio,a_o,tipo_de_crimen,lesionados_civiles,total_de_v_ctimas_del_caso
80,205166,EXTERIOR,FRONTERA PANAMA,77003,0,DESAPARICION_FORZADA,0.0,1.0
81,205167,EXTERIOR,FRONTERA PANAMA,77003,0,DESAPARICION_FORZADA,0.0,1.0
343,205577,ANTIOQUIA,SIN INFORMACION,05000,0,DESAPARICION_FORZADA,0.0,1.0
344,205579,CAQUETA,SIN INFORMACION,18000,0,DESAPARICION_FORZADA,0.0,1.0
345,205580,CAUCA,SIN INFORMACION,19000,0,DESAPARICION_FORZADA,0.0,1.0


In [87]:
# Verificamos categoria de municipios con codigos errados
registers_confirm['municipio'].value_counts()

,count
municipio,
SIN INFORMACION,1782
FRONTERA VENEZUELA,151
FRONTERA ECUADOR,33
MAPIRIPANA,29
FRONTERA PANAMA,24
BELEN DE BAJIRA,6
FRONTERA,3
FRONTERA BRASIL,2
FRONTERA PERU,1


Nota: No existe informacion sobre el municipio o corresponden a hechos fuera de colombia, salvo  Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio Barranco Minas de Guania, y Belen de Bajira, que en la bd figura como corregimiento de Mutata. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece y borramos los registros que corresponden al exterior y los que no se tiene informacion

In [88]:
# Verificar el registro con que aparecen
maripana = registers_confirm[registers_confirm['municipio']=='MAPIRIPANA']
print(maripana['c_digo_dane_de_municipio'].unique())


belen = registers_confirm[registers_confirm['municipio']=='BELEN DE BAJIRA']
print(belen['c_digo_dane_de_municipio'].unique())

['94663']
['27086']


In [89]:
# Buscamos los codigos de los municipios a los que pertenecen
print(dept_mpios_codes[dept_mpios_codes['mupio_name'] == 'BARRANCOMINAS'])
dept_mpios_codes[dept_mpios_codes['mupio_name'] == 'MUTATA']

     dept_mpio_code dept_name     mupio_name
1093          94343   GUAINIA  BARRANCOMINAS


,dept_mpio_code,dept_name,mupio_name
77,05480,ANTIOQUIA,MUTATA


In [90]:
# Reemplazar
cases_armed_conflict_sievcac['c_digo_dane_de_municipio'] = cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].replace( '94663', '94343')
cases_armed_conflict_sievcac['c_digo_dane_de_municipio'] = cases_armed_conflict_sievcac['c_digo_dane_de_municipio'].replace( '27086', '05480')

In [91]:
# Eliminamos los otros registros que corresponden al exterior y los sin informacion
# Definir las categorías a eliminar
delete_cat = ['SIN INFORMACION', 'FRONTERA VENEZUELA','FRONTERA ECUADOR','FRONTERA PANAMA','FRONTERA PERU','FRONTERA BRASIL','FRONTERA']

# Filtrar el DataFrame para eliminar las categorías especificadas
cases_armed_conflict_sievcac = cases_armed_conflict_sievcac[~cases_armed_conflict_sievcac['municipio'].isin(delete_cat)]

- Volvemos a veridfcar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [92]:
# Comparar listas de códigos
compare_lists(cases_armed_conflict_sievcac['c_digo_dane_de_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en cases_armed_conflict_sievcac", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en cases_armed_conflict_sievcac que no están en Códigos de municipios dept_mpios_codes:
set()


In [93]:
# Eliminar columnas 'departamento' y 'municipio' que ya no son necesarias
cases_armed_conflict_sievcac = cases_armed_conflict_sievcac.drop(['departamento', 'municipio'], axis=1)
cases_armed_conflict_sievcac.head(2)

,id_caso,c_digo_dane_de_municipio,a_o,tipo_de_crimen,lesionados_civiles,total_de_v_ctimas_del_caso
0,100265,25368,1991,DESAPARICION_FORZADA,0.0,1.0
1,100282,25386,2004,DESAPARICION_FORZADA,0.0,1.0


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [94]:
cases_armed_conflict_sievcac['tipo_de_crimen'].unique()

array(['DESAPARICION_FORZADA', 'ATENTADO_TERRORISTA', 'SECUESTRO',
       'ASESINATOS_SELECTIVOS', 'MINAS', 'VIOLENCIA_SEXUAL',
       'DANO_BIENES_CIVILES', 'MASACRES', 'ACCIONES_BELICAS',
       'ATAQUES_A_POBLADOS', 'RECLUTAMIENTO_NINOS'], dtype=object)

In [95]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
sources_id = [
    ('DESAPARICION_FORZADA', 98),
    ('ATENTADO_TERRORISTA', 99),
    ('SECUESTRO', 100),
    ('ASESINATOS_SELECTIVOS', 101),
    ('MINAS', 102),
    ('VIOLENCIA_SEXUAL', 103),
    ('DANO_BIENES_CIVILES', 104),
    ('MASACRES', 105),
    ('ACCIONES_BELICAS', 106),
    ('ATAQUES_A_POBLADOS', 107),
    ('RECLUTAMIENTO_NINOS', 108)
]

# Convertir la lista de tuplas en un diccionario para facilitar el mapeo
sources_dict = dict(sources_id)

# Crear la columna 'source_id' asignando el valor condicionalmente
cases_armed_conflict_sievcac['source_id'] = cases_armed_conflict_sievcac['tipo_de_crimen'].map(sources_dict)


In [96]:
# Agrupar el DataFrame por la columna 'source_id' y obtener el primer elemento de cada grupo
proof = cases_armed_conflict_sievcac.groupby('source_id').first()

# Mostrar los resultados
print(proof)

           id_caso c_digo_dane_de_municipio   a_o         tipo_de_crimen  \
source_id                                                                  
98          100265                    25368  1991   DESAPARICION_FORZADA   
99          185266                    11001  1977    ATENTADO_TERRORISTA   
100         102517                    27001  2005              SECUESTRO   
101         100000                    25290  2003  ASESINATOS_SELECTIVOS   
102         113698                    50325  2012                  MINAS   
103         359998                    86568     0       VIOLENCIA_SEXUAL   
104         121948                    54820  2002    DANO_BIENES_CIVILES   
105         100649                    25530  1998               MASACRES   
106              1                    27073  2006       ACCIONES_BELICAS   
107          10902                    27073  1997     ATAQUES_A_POBLADOS   
108         207556                    05756  2001    RECLUTAMIENTO_NINOS   

           

In [97]:
cases_armed_conflict_sievcac.columns

Index(['id_caso', 'c_digo_dane_de_municipio', 'a_o', 'tipo_de_crimen',
       'lesionados_civiles', 'total_de_v_ctimas_del_caso', 'source_id'],
      dtype='object')

In [98]:
# Crear el diccionario manualmente
column_mapping = {
    'id_caso': 'id_sievcac',
    'c_digo_dane_de_municipio': 'dane_code',
    'a_o': 'year',
    'tipo_de_crimen': 'type_of_crime',
    'lesionados_civiles':'injured_civilians',
    'total_de_v_ctimas_del_caso':'total_number_of_victims',
    'source_id': 'source_id'
}

# Renombrar las columnas del DataFrame
cases_armed_conflict_sievcac = cases_armed_conflict_sievcac.rename(columns=column_mapping)

In [99]:
#Estructura final del dataset a integrar a la base de datos
cases_armed_conflict_sievcac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 349503 entries, 0 to 360219
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id_sievcac               349503 non-null  int64  
 1   dane_code                349503 non-null  object 
 2   year                     349503 non-null  int64  
 3   type_of_crime            349503 non-null  object 
 4   injured_civilians        349503 non-null  float64
 5   total_number_of_victims  349503 non-null  float64
 6   source_id                349503 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 21.3+ MB


In [100]:
cases_armed_conflict_sievcac.head()

,id_sievcac,dane_code,year,type_of_crime,injured_civilians,total_number_of_victims,source_id
0,100265,25368,1991,DESAPARICION_FORZADA,0.0,1.0,98
1,100282,25386,2004,DESAPARICION_FORZADA,0.0,1.0,98
2,101616,25885,1993,DESAPARICION_FORZADA,0.0,2.0,98
3,102675,27615,1997,DESAPARICION_FORZADA,0.0,1.0,98
4,103705,41396,2006,DESAPARICION_FORZADA,0.0,2.0,98


## Salvar en archivo csv en el drive

In [101]:
# Guardar en archivos CSV en el drive
cases_armed_conflict_sievcac.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/cases_armed_conflict_sievcac.csv', index=False)